# Secbert를 위한 arxiv 데이터 준비 & 크롤링
190718

2015년부터 ,arxiv에서 security 관련된 모든 논문(만개)들의

title, abstract를 모아서 bert의 입력형태인

    <<doc>>
    title
    abstract

형태로 저장한다

- 크롤링 소스
https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=security&terms-0-field=all&classification-computer_science=y&classification-eess=y&classification-physics_archives=all&classification-include_cross_list=include&date-year=&date-filter_by=date_range&date-from_date=2015&date-to_date=&date-date_type=submitted_date&abstracts=hide&size=50&order=-announced_date_first&start=9950

- 방법 ref
https://github.com/aisolab/simple_examples_for_text_mining/blob/master/Scrapping%20text%20mining%20papers%20in%20arXiv.py

- Error시 참고
    크롤링을 돌리다 보면, arxiv쪽에서 공격으로 간주하고 ip접근을 차단한다.
    그러므로 vpn(nodevpn)을 사용하여 ip를 우회하여 크롤링하고,
    끊기면 다시 ip를 바꿔 이어 시작하는 형태로 한다.
    

In [1]:
import requests
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
import string
import datetime
from datetime import datetime
import random

In [2]:
save_output = 'security_arxiv_paper.txt'

In [3]:
# URL 작성
base_url = 'https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=security&terms-0-field=all&classification-computer_science=y&classification-eess=y&classification-physics_archives=all&classification-include_cross_list=include&date-year=&date-filter_by=date_range&date-from_date=2015&date-to_date=&date-date_type=submitted_date&abstracts=hide&size=50&order=-announced_date_first'
# 페이지 넘어가며 뒤에 붙는 주소
url_list = ['']
temp = list(range(50, 10000, 50))  # '', '&start=50', '&start=100', ...8950 -> 총 10,106개
for i in temp: url_list.append('&start={}'.format(i))

In [4]:
def printable_text_true(text):
    # printable 하면 False
    # 이상한게 있으면 True
    not_printable = 0
    for char in text:
        if char not in string.printable:
            not_printable += 1
    if not_printable == 0:
        return False
    else:
        return True

In [10]:
# 처음 시작할땐 text 을 write 모드로 쓰기 위해 'w', 
# ip차단되어 vpn으로 ip바꾸고 다시 접속할땐 기존 text에 덮어쓰기 위해 'a'
start_index = 178
save_text = 'w' if start_index == 0 else 'a'

file = open(save_output, save_text, encoding='UTF8')
refused_wait_time = 60*1

start = time.clock()
count_doc, count_pass = 0, 0
for i in range(start_index, len(url_list)):
    print('%d / %d - %d / %d, %.19s' %(i, len(url_list), count_doc, count_pass, datetime.now()))
    
    tmp_url = base_url + url_list[i]  # 페이지 넘어가는 url 만듬
    
    ## 막히는 경우 예외처리
    tmp_response = ''
    while tmp_response == '':
        try:
            tmp_response = requests.get(tmp_url)
            wait_time = random.randrange(8,11)
            time.sleep(wait_time)
            break
        except:
            print("Connection refused, wait {} sec".format(refused_wait_time))
            time.sleep(refused_wait_time)
            continue            
    
    tmp_bs = BeautifulSoup(tmp_response.text, 'html.parser')
    tmp_contents = tmp_bs.select('li.arxiv-result')
    tmp_list = [tmp_href.select_one('a').attrs.get('href') for tmp_href in tmp_contents]

    # ['https://arxiv.org/abs/1906.11488',
    #  'https://arxiv.org/abs/1906.11465', ...]

    for j, tmp_paper in enumerate(tmp_list):
        # tmp_paper : archive 주소!
        tmp_response_paper = requests.get(tmp_paper)  # <Response [200]>
        tmp_bs_paper = BeautifulSoup(tmp_response_paper.text, 'html.parser')

        title = re.sub('\\s+',
                         ' ',
                         tmp_bs_paper.select_one('h1.title.mathjax').text.replace('Title:', ''))
        abstract = tmp_bs_paper.select_one('blockquote.abstract.mathjax').text.replace('Abstract:  ', '').replace('\n', ' ')

        if '$' in abstract or '$' in title or printable_text_true(title) or printable_text_true(abstract):
            count_pass += 1
            pass
        # 다 printable 하면
        else:
            # 영어, 숫자, () 빼고 다 삭제
            title = re.sub('[^a-zA-Z1-9()%\' -%&,.]', '', title)
            abstract = re.sub('[^a-zA-Z1-9()%\' -%&,.]', '', abstract)
            
            file.write("<<doc>>\n")
            file.write(title+'\n')
            file.write(abstract+'\n\n')
            count_doc += 1

end = time.clock()


print('%d sec'%(end - start))
print(count_doc)


178 / 200 - 0 / 0, 2019-07-18 19:06:28
Connection refused, wait 60 sec
179 / 200 - 41 / 9, 2019-07-18 19:08:27
180 / 200 - 88 / 12, 2019-07-18 19:09:25
181 / 200 - 132 / 18, 2019-07-18 19:10:25
182 / 200 - 180 / 20, 2019-07-18 19:11:26
183 / 200 - 222 / 28, 2019-07-18 19:12:26
184 / 200 - 268 / 32, 2019-07-18 19:13:29
185 / 200 - 316 / 34, 2019-07-18 19:14:31
186 / 200 - 358 / 42, 2019-07-18 19:15:35
187 / 200 - 405 / 45, 2019-07-18 19:16:36
188 / 200 - 450 / 50, 2019-07-18 19:17:38
189 / 200 - 495 / 55, 2019-07-18 19:18:42
190 / 200 - 539 / 61, 2019-07-18 19:19:43
191 / 200 - 579 / 71, 2019-07-18 19:20:46
192 / 200 - 617 / 83, 2019-07-18 19:21:48
193 / 200 - 660 / 90, 2019-07-18 19:22:51
194 / 200 - 707 / 93, 2019-07-18 19:23:52
195 / 200 - 751 / 99, 2019-07-18 19:24:55
196 / 200 - 795 / 105, 2019-07-18 19:26:05
197 / 200 - 839 / 111, 2019-07-18 19:27:08
198 / 200 - 884 / 116, 2019-07-18 19:28:14
199 / 200 - 928 / 122, 2019-07-18 19:29:17
1430 sec
965
